# X-Ray

Imports


In [ ]:
import os
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt

### Data check

In [ ]:

# Path to the dataset
dataset_path = 'data/raw/covid-chest-xray'

# Dictionary to store image counts per folder
folder_image_counts = {}

# Iterate through each subdirectory
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    
    # Check if it's a directory
    if os.path.isdir(folder_path):
        # List all files in the folder
        all_files = os.listdir(folder_path)
        
        # Filter image files (assuming they have common image extensions)
        image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Count the number of images in the folder
        folder_image_counts[folder] = len(image_files)

# Print the image counts per folder
for folder, count in folder_image_counts.items():
    print(f"{folder}: {count} images")

Ignoramos bacterial o hacemos data-augmentation para balancear las clases?


Generamos carpeta con las imagenes de train y test


In [ ]:
# Path to the original dataset and output directory
dataset_path = "data/raw/covid-chest-xray"
output_path = "data/split/covid-chest-xray"

# Define train-test split ratio
train_ratio = 0.8

# Create train and test directories
train_path = os.path.join(output_path, "train")
test_path = os.path.join(output_path, "test")

# Ensure directories exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Iterate through each category (subfolder)
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    
    # Check if it's a directory
    if os.path.isdir(category_path):
        # Create corresponding train/test subfolders
        train_category_path = os.path.join(train_path, category)
        test_category_path = os.path.join(test_path, category)
        os.makedirs(train_category_path, exist_ok=True)
        os.makedirs(test_category_path, exist_ok=True)
        
        # Get all image files
        all_images = [f for f in os.listdir(category_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(all_images)  # Shuffle to randomize selection

        # Split data
        train_count = int(len(all_images) * train_ratio)
        train_images = all_images[:train_count]
        test_images = all_images[train_count:]

        # Move images to respective folders
        for img in train_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(train_category_path, img))
        
        for img in test_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(test_category_path, img))

        print(f"Processed {category}: {len(train_images)} train, {len(test_images)} test")

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'data/split/covid-chest-xray/train',
    target_size=(150, 150),
    batch_size=25,
    class_mode='binary') #class_mode='categorical' cambiar si metemos bacterial

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    'data/split/covid-chest-xray/test',
    target_size=(150, 150),
    batch_size=25,
    class_mode='binary') #class_mode='categorical' cambiar si metemos bacterial

In [ ]:
# Load the VGG16 model
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = True

In [ ]:
for layer in conv_base.layers:
    if layer.name.startswith('block5'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
conv_base.summary()

In [ ]:
# Create a new model and add the VGG16 base
model = models.Sequential()
model.add(conv_base)

# Add new layers on top of the VGG16 base
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=50, #100
    epochs=15, #30
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
history_dict = history.history

In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# ResNet50

## Cargar el modelo ResNet50

In [ ]:
# Load the ResNet50 model
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = True

In [ ]:
for layer in conv_base.layers:
    if layer.name.startswith('conv5'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
conv_base.summary()

## Adaptar el modelo

In [ ]:
# Create a new model and add the ResNet50 base
model = models.Sequential()
model.add(conv_base)

# Add new layers on top of the ResNet50 base
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=50, #100
    epochs=15, #30
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
history_dict = history.history

## Resultados

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()